## Instalação e importação de libs e packs

In [2]:
# 📄 setup_environment.py
# Instalações básicas (executar apenas uma vez)
!pip install geopandas rasterio folium matplotlib osmnx googlemaps ee sentinelsat
!jupyter nbextension enable --py widgetsnbextension

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.8 MB/s eta 0:00:00
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=f577f5acddf8fd679fd8b7b40d1eda9b93e825af42d1086ab1a65d224ae075fb
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
  Created wheel for ee: filename=ee-0.2-py3-none-any.whl size=3655 sha256=8dc61f1a5337b3ad19b14e7a6c6e55887bf7cc8fb82ef6c9b402211a98607002
  Stored in directory: /root/.cache/pip/wheels/a0/6d/ee/1d25db5c1257f6b86266fa4c20a2439110c42e8759486066f7
Successfully built googlemaps ee
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/note

In [3]:
# 📄 0_user_input_map.py
import folium
import math
import json
import html
from IPython.display import display, clear_output, HTML
from ipywidgets import widgets, Output, Layout
from branca.element import MacroElement, Figure, Html
from jinja2 import Template

## Entrada de dados e captura do click no mapa

In [4]:
# 📌 Célula 1 - Entrada dos Raios

# Widgets para entrada dos raios
raio_aid = widgets.IntText(
    value=500,
    min=100,
    max=2000,
    step=50,
    description='Raio AID (m):',
    layout=Layout(width='300px')
)

raio_aii = widgets.IntText(
    value=1000,
    min=500,
    max=5000,
    step=100,
    description='Raio AII (m):',
    layout=Layout(width='300px')
)

display(widgets.VBox([raio_aid, raio_aii]))

print("Defina os raios de influência acima e execute a próxima célula")

Defina os raios de influência acima e execute a próxima célula


In [5]:
# 📌 Célula Única - Captura de Localização Simplificada

def criar_mapa_selecao():
    """Cria mapa interativo apenas para seleção do local do empreendimento."""

    # Cria mapa centrado nas coordenadas padrão
    m = folium.Map(location=[-15.61081, -56.07658], zoom_start=16)

    # Variável para armazenar coordenadas
    empreendimento_coords = None

    def adicionar_marcador(**kwargs):
        """Adiciona marcador quando usuário clica no mapa."""
        nonlocal empreendimento_coords

        if kwargs.get('lat') and kwargs.get('lng'):
            # Remove marcadores anteriores
            for item in m._children.copy():
                if isinstance(m._children[item], folium.Marker):
                    m._children.pop(item)

            # Adiciona novo marcador
            empreendimento_coords = (kwargs['lat'], kwargs['lng'])
            folium.Marker(
                location=empreendimento_coords,
                popup=f"Empreendimento<br>Lat: {kwargs['lat']:.6f}<br>Lng: {kwargs['lng']:.6f}",
                icon=folium.Icon(color='red')
            ).add_to(m)

    # Configura interação
    m.add_child(folium.LatLngPopup())  # Mostra coordenadas ao clicar
    m.add_child(folium.ClickForMarker(popup="Clique para posicionar"))

    # JavaScript customizado para garantir funcionamento no Colab
    click_js = """
    <script>
    function getMap() {
        return window[document.currentScript.parentElement.id];
    }

    document.addEventListener('DOMContentLoaded', function() {
        var map = getMap();
        map.on('click', function(e) {
            var marker = L.marker(e.latlng, {
                icon: L.icon({
                    iconUrl: 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/1.7.1/images/marker-icon.png',
                    iconSize: [25, 41],
                    iconAnchor: [12, 41]
                })
            }).addTo(map);
            marker.bindPopup("Local selecionado").openPopup();
        });
    });
    </script>
    """
    m.get_root().html.add_child(folium.Element(click_js))

    # Exibe o mapa
    display(m)
    print("Clique no mapa para selecionar a localização do empreendimento")

    return empreendimento_coords

# Uso:
mapa = criar_mapa_selecao()

Clique no mapa para selecionar a localização do empreendimento


In [6]:
print(mapa)  # deveria aparecer as coordenadas capturadas...

None
